In [9]:
import pandas as pd
import re
import xml.etree.ElementTree as ET
import zipfile
from collections import Counter

import numpy as np
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
#from collections import Counter
import math

In [14]:
df = pd.read_csv("../data_Swan/original_data.tsv", delimiter="\t")
df
# project_fk   => プロジェクト別 , 今プロジェクトが3つあるからこうなってる

,id,name,text,project_fk
0,1,HW1_pm,Hello everyone. Today’s topic is “Homework sho...,1
1,15,LO1-1,Hello everyone. Today’s topic is “Homework sho...,6
2,27,LO1-4,Hello everyone. Today’s topic is “Homework sho...,6
3,25,LO1-2,Hello everyone. Today’s topic is “Homework sho...,6
4,26,LO1-3,Hello everyone. Today’s topic is “Homework sho...,6
5,28,LO1-6,Hello everyone. Today’s topic is “Homework sho...,6
6,29,LO1-2,Hello everyone. Today’s topic is “Homework sho...,7
7,30,LO1-1,Hello everyone. Today’s topic is “Homework sho...,7
8,33,LO1-3,Hello everyone. Today’s topic is “Homework sho...,7
9,31,LO1-4,Hello everyone. Today’s topic is “Homework sho...,7


In [15]:
df = df[df["project_fk"] == 7]   # これが今使ってるプロジェクトのファイル
df

,id,name,text,project_fk
6,29,LO1-2,Hello everyone. Today’s topic is “Homework sho...,7
7,30,LO1-1,Hello everyone. Today’s topic is “Homework sho...,7
8,33,LO1-3,Hello everyone. Today’s topic is “Homework sho...,7
9,31,LO1-4,Hello everyone. Today’s topic is “Homework sho...,7
10,32,LO1-6,Hello everyone. Today’s topic is “Homework sho...,7
11,34,LO1-8,Hello everyone. Today’s topic is “Homework sho...,7
12,35,LO1-5,Hello everyone. Today’s topic is “Homework sho...,7
13,37,LO1-7,Hello everyone. Today’s topic is “Homework sho...,7
14,38,LO1-10,Hello everyone. Today’s topic is “Homework sho...,7
15,39,LO1-11,Hello everyone. Today’s topic is “Homework sho...,7


In [16]:
 with zipfile.ZipFile('../data_Swan/export_ArgGraph_.zip', 'r') as zip_data:
    filename_list = zip_data.namelist()
filename_list

['ArgGraph__LO1-6_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-6_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-3_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-3_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-8_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-8_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-5_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-5_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-7_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-7_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-10_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-10_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-11_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-11_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-12_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-12_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-14_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-14_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-9_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-9_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-13_ano1.graph@swan.jp.xml',
 'ArgGraph__LO1-13_ano2.graph@swan.jp.xml',
 'ArgGraph__LO1-15_ano1.graph@swan.jp.xml',


In [17]:
def get_span_text_info(original_text, xml):
    root = ET.fromstring(xml)
    # get span_text
    span_text_dict = {}
    LO_span_texts_list = []
    for value in root.iter('annotation'):
        span_id = int(value.find('id').text)
        start = int(value.find('start').text)
        end = int(value.find('end').text)
        span_text = original_text[start:end]
        span_text = re.sub(r'[,.!?:;"]', '', span_text).lower()
        # ready for relation & FB
        span_text_dict[span_id] = span_text
        # ready for word_freqs
        span = value.find('spanType')
        span_type = (span.text if span is not None else '')
        if span_type == 'LO':
            LO_span_texts_list += span_text.split(" ")
            
    return span_text_dict, LO_span_texts_list

In [18]:
def get_FB_info(span_text_dict, xml):
    root = ET.fromstring(xml)
    FB_dict = {}   # {to_id : FB_text}
    for value in root.iter('link'):
        from_id = int(value.find('from').text)
        to_id = int(value.find('to').text)
        label = value.find('.//label')
        relation_type = (label.text if label is not None else '')
        fb_info = ["GP", "BP", "GP-edge", "BP-edge"]
        if relation_type in fb_info:
            FB_dict[to_id] = span_text_dict[from_id]
            continue
        else:
            continue
    return FB_dict

In [19]:
def ready_KB(span_text_dict, FB_dict, xml):
    root = ET.fromstring(xml)
    KB_data = []
    for value in root.iter('link'):
        FB = []
        from_id = int(value.find('from').text)
        to_id = int(value.find('to').text)
        label = value.find('.//label')
        relation_type = (label.text if label is not None else '')

        fb_info = ["GP", "BP", "GP-edge", "BP-edge"]
        if relation_type in fb_info:
            continue
        else:
            if from_id in FB_dict:
                if to_id in FB_dict:
                    FB.append(FB_dict[from_id])
                    FB.append(FB_dict[to_id])
                else:
                    FB.append(FB_dict[from_id])
            elif to_id in FB_dict:
                FB.append(FB_dict[to_id])
            else:
                FB.append("None")
            #前のが記憶されている？
        KB_data.append([span_text_dict[from_id], span_text_dict[to_id], relation_type, ",".join(FB)])
    
    return KB_data

In [20]:
LO_span_texts_lists = []
datasets = []
with zipfile.ZipFile('../data_Swan/export_ArgGraph_.zip', 'r') as zip_data:
    filename_list = zip_data.namelist()
    for filename in filename_list:
        m = re.match('ArgGraph__(.*)_(.*).graph@swan.jp.xml', filename)
        if not m:
            continue
        if m.group(2) == "ano2":
            continue       # pick up only ano1_data
        debate_id = m.group(1)
        df = pd.read_csv('../data_Swan/original_data.tsv', delimiter='\t')
        original_text = (df[df['name'] == debate_id])[:1]['text'].values[0]
        xml = zip_data.read(filename)
        span_text_dict, LO_span_texts_list = get_span_text_info(original_text, xml)
        LO_span_texts_lists += LO_span_texts_list
        FB_dict = get_FB_info(span_text_dict, xml)
        KB_data = ready_KB(span_text_dict, FB_dict, xml)
        datasets += KB_data

word_freqs = Counter(LO_span_texts_lists)
print(datasets)
print(span_text_dict)
print(FB_dict)

[['they may not always aquire something useful in their life from such activities', 'if what they really want to do is such a less useful activity they should do it current free time for example after finishing their homework', 'support', 'いかに”less useful activity”であるのか、説明できるとよりよいです'], ['they may not always aquire something useful in their life from such activities', 'we could do more of what we really wanted like club activities hobbies or playing with friends', 'undercut', 'None'], ['being aware that when they finish thier homework they can do what they want to do they can concentrate on their homework to finish it quickly', 'they can study efficiently', 'support', 'None'], ['they can study efficiently', 'we believe that if homework were to be abolished we could have more free time', 'undercut', 'None'], ['students can enter their ideal university and learn knowledge is inseparable', "i don't think homework should be abolished", 'support', 'None'], ['we can reduce the amount of homew

In [21]:
KB_df = pd.DataFrame(datasets)
KB_df

,0,1,2,3
0,they may not always aquire something useful in...,if what they really want to do is such a less ...,support,いかに”less useful activity”であるのか、説明できるとよりよいです
1,they may not always aquire something useful in...,we could do more of what we really wanted like...,undercut,None
2,being aware that when they finish thier homewo...,they can study efficiently,support,None
3,they can study efficiently,we believe that if homework were to be abolish...,undercut,None
4,students can enter their ideal university and ...,i don't think homework should be abolished,support,None
5,we can reduce the amount of homework properly,i don't think homework should be abolished,support,宿題を適切に減らすとよい、と述べられていましたが、どのように適切に減らせるか説明できるとより...
6,let students have more free time,we can reduce the amount of homework properly,support,宿題を適切に減らすとよい、と述べられていましたが、どのように適切に減らせるか説明できるとより...
7,there are many students in many countries in t...,they don't have time to participate other acti...,support,None
8,we can reduce the amount of homework properly,if homework were to be abolished we could have...,undercut,宿題を適切に減らすとよい、と述べられていましたが、どのように適切に減らせるか説明できるとより...
9,homework is a kind of review and feedback for ...,students can enter their ideal university and ...,support,宿題の特徴分析として、宿題は復習としてその日にやるからこそ重要と述べられた点は、よかったです


In [29]:
from_text_data_list = [data_info[0].split() for data_info in datasets]
from_text_data_list


[['they',
  'may',
  'not',
  'always',
  'aquire',
  'something',
  'useful',
  'in',
  'their',
  'life',
  'from',
  'such',
  'activities'],
 ['they',
  'may',
  'not',
  'always',
  'aquire',
  'something',
  'useful',
  'in',
  'their',
  'life',
  'from',
  'such',
  'activities'],
 ['being',
  'aware',
  'that',
  'when',
  'they',
  'finish',
  'thier',
  'homework',
  'they',
  'can',
  'do',
  'what',
  'they',
  'want',
  'to',
  'do',
  'they',
  'can',
  'concentrate',
  'on',
  'their',
  'homework',
  'to',
  'finish',
  'it',
  'quickly'],
 ['they', 'can', 'study', 'efficiently'],
 ['students',
  'can',
  'enter',
  'their',
  'ideal',
  'university',
  'and',
  'learn',
  'knowledge',
  'is',
  'inseparable'],
 ['we', 'can', 'reduce', 'the', 'amount', 'of', 'homework', 'properly'],
 ['let', 'students', 'have', 'more', 'free', 'time'],
 ['there',
  'are',
  'many',
  'students',
  'in',
  'many',
  'countries',
  'in',
  'the',
  'world',
  'whose',
  'homework',
  'is

In [22]:
word_freqs

Counter({'if': 3,
         'what': 7,
         'they': 13,
         'really': 1,
         'want': 4,
         'to': 20,
         'do': 11,
         'is': 21,
         'such': 3,
         'a': 7,
         'less': 1,
         'useful': 2,
         'activity': 2,
         'should': 3,
         'it': 4,
         'current': 1,
         'free': 6,
         'time': 13,
         'for': 7,
         'example': 1,
         'after': 2,
         'finishing': 1,
         'their': 10,
         'homework': 20,
         'being': 1,
         'aware': 1,
         'that': 3,
         'when': 1,
         'finish': 2,
         'thier': 1,
         'can': 11,
         'concentrate': 1,
         'on': 2,
         'quickly': 1,
         'study': 9,
         'efficiently': 2,
         'may': 2,
         'not': 4,
         'always': 1,
         'aquire': 1,
         'something': 1,
         'in': 7,
         'life': 2,
         'from': 1,
         'activities': 2,
         'i': 4,
         "don't": 4,
         '

ここから　SIF

In [4]:
PATH_TO_GLOVE = "../data_model/glove.840B.300d.txt"

tmp_file = "../data_model/glove.840B.300d.w2v.txt"
glove2word2vec(PATH_TO_GLOVE, tmp_file)
glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)
glove

In [ ]:
tmp_file = "../data_model/glove.840B.300d.w2v.txt"
glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)
glove

In [7]:
# 確認
glove["what"]

array([-3.8548e-02,  5.4252e-01, -2.1843e-01, -1.8855e-01,  7.3000e-02,
        1.3180e-01, -1.0402e-01,  1.7231e-01, -5.1587e-02,  2.8646e+00,
       -2.5131e-01, -1.9146e-01,  7.9987e-02,  7.2892e-02, -1.3209e-01,
       -2.2971e-01, -1.7268e-01,  8.3507e-01, -3.8487e-01, -3.2492e-01,
        4.6680e-01, -1.5678e-01,  1.0707e-01, -1.2479e-01, -1.6030e-01,
       -5.2154e-03,  1.5166e-02,  1.6508e-01,  9.8025e-02, -9.1898e-02,
        2.4552e-04,  4.4254e-01,  1.6046e-01,  1.2215e-01,  1.2594e-01,
        3.1999e-01,  2.2045e-01,  1.6870e-01, -3.8927e-01, -1.7347e-01,
       -1.3158e-01,  7.9817e-02, -5.8053e-02, -7.2235e-02,  2.8033e-01,
        6.1080e-03, -3.9289e-01,  6.1744e-04, -3.3363e-02,  7.5885e-03,
       -9.3664e-02,  1.8932e-01, -2.2542e-01, -1.8091e-02,  2.9546e-01,
       -1.5163e-01, -1.5594e-01, -2.1939e-01,  8.8317e-02, -6.5472e-02,
       -6.2950e-02, -1.9989e-01, -4.9454e-03,  2.9574e-01,  2.7333e-01,
       -3.6213e-01, -4.0510e-02,  2.2045e-01,  8.4721e-02,  6.27

In [10]:
# np 計算確認
np.average(glove["what"])

-0.0016508654

In [53]:
def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX

# SIFモデル
def calculate_similarities_sif(input_text, KB_texts, model, freqs={}, a=0.001): 
    total_freq = sum(freqs.values())
    
    embeddings = []
    
    # SIF requires us to first collect all sentence embeddings and then perform 
    # common component analysis.
    for KB_text in KB_texts:
        input_tokens = [token for token in input_text if token in model]
        KB_tokens = [token for token in KB_text if token in model]
        
        weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in input_tokens]
        weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in KB_tokens]
        # numpy型にしておく必要あり?
        embedding1 = np.average([model[token] for token in input_tokens], axis=0, weights=weights1)
        embedding2 = np.average([model[token] for token in KB_tokens], axis=0, weights=weights2)
        
        embeddings.append(embedding1)
        embeddings.append(embedding2)
        
    embeddings = remove_first_principal_component(np.array(embeddings))
    sims = [cosine_similarity(embeddings[idx*2].reshape(1, -1), 
                              embeddings[idx*2+1].reshape(1, -1))[0][0] 
            for idx in range(int(len(embeddings)/2))]

    return np.array(sims)


def multiplication_sims(input_data, KB_data, model, freqs):
    # set new data
    input_1 = re.sub(r'[,.!?:;"]', '', input_data[0]).lower().split()
    input_2 = re.sub(r'[,.!?:;"]', '', input_data[1]).lower().split()
    # set KB data
    from_text_data_list = [data_info[0].split() for data_info in KB_data]
    to_text_data_list = [data_info[1].split() for data_info in KB_data]
    # calculate
    from_sims = calculate_similarities_sif(input_1, from_text_data_list, model, freqs)
    to_sims = calculate_similarities_sif(input_2, to_text_data_list, model, freqs)
    
    multi_sims = from_sims * to_sims
    return np.max(multi_sims), np.argmax(multi_sims)

In [47]:
# set data
input_data = ["if you do homework efficiently, you will have more free time", 
              "There is no reason homework should be abolished"]
input_1 = re.sub(r'[,.!?:;"]', '', input_data[0]).lower().split()
input_2 = re.sub(r'[,.!?:;"]', '', input_data[1]).lower().split()
print(input_1)
print(input_2)

['if', 'you', 'do', 'homework', 'efficiently', 'you', 'will', 'have', 'more', 'free', 'time']
['there', 'is', 'no', 'reason', 'homework', 'should', 'be', 'abolished']


In [55]:
from_text_data_list = [data_info[0].split() for data_info in datasets]
to_text_data_list = [data_info[1].split() for data_info in datasets]
from_sims = calculate_similarities_sif(input_1, from_text_data_list, glove, word_freqs)
to_sims = calculate_similarities_sif(input_2, to_text_data_list, glove, word_freqs)

multi_sims = from_sims * to_sims
max_value = np.max(multi_sims)
max_index = np.argmax(multi_sims)

print(max_value, max_index)

0.4212476078205923 40


In [56]:
datasets[40]

['what we learn at school is very significant and we often apply that knowledge to our favorite things to do',
 'the most important work for students is study',
 'support',
 '生徒の一番の仕事は勉強、とスタンスをクリアにしたことがよかったです']

In [54]:
value, index = multiplication_sims(input_data, datasets, glove, word_freqs)
datasets[index]

['what we learn at school is very significant and we often apply that knowledge to our favorite things to do',
 'the most important work for students is study',
 'support',
 '生徒の一番の仕事は勉強、とスタンスをクリアにしたことがよかったです']

以下　道中のメモ

In [ ]:
def get_ADU_dict(original_text, xml):
    root = ET.fromstring(xml)
    span_text_dict = {}
    for value in root.iter('annotation'):
        span_id = value.find('id').text
        start = int(value.find('start').text)
        end = int(value.find('end').text)
        span_text = original_text[start:end]
        span_text_dict[span_id] = span_text
    return span_text_dict

def setup_data(span_text_dict, xml):
    data = []
    root = ET.fromstring(xml)
    for value in root.iter('link'):
        from_id = value.find('from').text
        to_id = value.find('to').text
        label = value.find('.//label')
        relation_type = (label.text if label is not None else '')
        data.append((span_text_dict[from_id], span_text_dict[to_id], relation_type))
    return data

In [8]:
ADU_lists = []
datasets = []
with zipfile.ZipFile('./export_ArgGraph_.zip', 'r') as zip_data:
    filename_list = zip_data.namelist()
    for filename in filename_list:
        m = re.match('ArgGraph__(.*)_(.*).graph@swan.jp.xml', filename)
        if not m:
            continue
        if m.group(2) == "ano2":
            continue       # pick up only ano1_data
        debate_id = m.group(1)
        df = pd.read_csv('./original_data.tsv', delimiter='\t')
        original_text = (df[df['name'] == debate_id])[:1]['text'].values[0]
        xml = zip_data.read(filename)
        ADU_dict = get_ADU_dict(original_text, xml)
        ADU_lists.append(ADU_dict)
        dataset = setup_data(ADU_dict, xml)
        datasets.append(dataset)
ADU_lists

[{'256': 'If what they really want to do is such a "less useful activity", they should do it current free time, for example after finishing their homework',
  '257': 'being aware that "when they finish thier homework, they can do what they want to do", they can concentrate on their homework to finish it quickly',
  '260': 'いかに”less useful activity”であるのか、説明できるとよりよいです',
  '261': 'We believe that if homework were to be abolished, we could have more free time',
  '255': 'we could do more of what we really wanted like club activities, hobbies, or playing with friends',
  '258': 'they can study efficiently',
  '259': '自由時間を”less useful activity”に使っていると強調できているのがよかったです',
  '254': 'they may not always aquire something useful in their life from such activities'},
 {'227': 'if homework were to be abolished, we could have more free time',
  '223': "I don't think homework should be abolished",
  '219': 'homework is a kind of review and feedback for students who have learned knowledge on the same da

In [9]:
len(ADU_lists)   # 42 = 21*2  orizinal_text数 × anno数   今はano1のみだから

21

In [10]:
ADU_lists[0]

{'256': 'If what they really want to do is such a "less useful activity", they should do it current free time, for example after finishing their homework',
 '257': 'being aware that "when they finish thier homework, they can do what they want to do", they can concentrate on their homework to finish it quickly',
 '260': 'いかに”less useful activity”であるのか、説明できるとよりよいです',
 '261': 'We believe that if homework were to be abolished, we could have more free time',
 '255': 'we could do more of what we really wanted like club activities, hobbies, or playing with friends',
 '258': 'they can study efficiently',
 '259': '自由時間を”less useful activity”に使っていると強調できているのがよかったです',
 '254': 'they may not always aquire something useful in their life from such activities'}

In [11]:
datasets

[[('they may not always aquire something useful in their life from such activities',
   'If what they really want to do is such a "less useful activity", they should do it current free time, for example after finishing their homework',
   'support'),
  ('自由時間を”less useful activity”に使っていると強調できているのがよかったです',
   'If what they really want to do is such a "less useful activity", they should do it current free time, for example after finishing their homework',
   'GP'),
  ('they may not always aquire something useful in their life from such activities',
   'we could do more of what we really wanted like club activities, hobbies, or playing with friends',
   'undercut'),
  ('いかに”less useful activity”であるのか、説明できるとよりよいです',
   'If what they really want to do is such a "less useful activity", they should do it current free time, for example after finishing their homework',
   'BP'),
  ('being aware that "when they finish thier homework, they can do what they want to do", they can concentrate on the